In [1]:
def dtype_compressor(df):
  #object 
  # when unique ratio > 0.5, catogory stands less memory than object
  converted_obj = pd.DataFrame()
  df_obj = df.select_dtypes(include=['object'])
  for col in df_obj.columns:
    converted_obj.loc[:,col] = df_obj.loc[:,col].astype('category')

  # numbers
  df_down_num = pd.DataFrame()
  df_num = df.select_dtypes(include=['number'])
  for col in df_num.columns:
    df_down_num.loc[:,col] = pd.to_numeric(df_num.loc[:,col],downcast='signed')

  # merge
  df = pd.concat([converted_obj,df_down_num],axis=1)
  
  del converted_obj, df_obj, df_down_num, df_num  
  
  return df

In [2]:
def fast_datetime(s):
    dates = {date:pd.to_datetime(date,format='%Y%m%d') for date in s.unique()}
    return s.map(dates)

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [4]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [49]:
KKboxTransac_1 = pd.read_csv("transactions.csv")
KKboxTransac_2 = pd.read_csv("transactions_v2.csv")

In [50]:
KKboxTransac = pd.concat([KKboxTransac_1, KKboxTransac_2], axis=0)

In [51]:
# 如有記憶體優化需求時可用
del KKboxTransac_1
del KKboxTransac_2

In [71]:
KKboxTransac.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,membership_days
6797850,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,35,7,0,0,0,0,5.000
1914756,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,28.000
1498592,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,29.000
17923235,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,31.000
58566,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,31.000


In [72]:
KKboxTransac.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22437106 entries, 6797850 to 9570304
Data columns (total 8 columns):
msno                  category
payment_method_id     int8
payment_plan_days     int16
plan_list_price       int16
actual_amount_paid    int16
is_auto_renew         int8
is_cancel             int8
membership_days       float64
dtypes: category(1), float64(1), int16(3), int8(3)
memory usage: 934.2 MB


In [73]:
KKboxTransac.describe()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,membership_days
count,22437106.000,22437106.000,22437106.000,22437106.000,22437106.000,22437106.000,22437106.000
mean,38.923,30.317,134.959,136.597,0.853,0.033,34.245
std,3.522,19.890,84.666,83.912,0.354,0.178,27.689
min,1.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,38.000,30.000,99.000,99.000,1.000,0.000,30.000
50%,41.000,30.000,149.000,149.000,1.000,0.000,31.000
75%,41.000,30.000,149.000,149.000,1.000,0.000,31.000
max,41.000,415.000,1825.000,1825.000,1.000,1.000,365.000


-------

##### 型態轉換處理(優化記憶體、日期轉換)

In [55]:
date_rename = {'transaction_date':'trans_date', 'membership_expire_date':'mem_expire_date'}
KKboxTransac.rename(columns=date_rename,inplace=True)

In [56]:
del date_rename

In [57]:
KKboxTransac = dtype_compressor(KKboxTransac)

In [58]:
d = ['trans_date', 'mem_expire_date']
for col in d:
  KKboxTransac[col] = fast_datetime(KKboxTransac[col])

In [59]:
KKboxTransac = KKboxTransac.sort_values(by=['msno','trans_date'])

---

##### 將mem_expire_date - trans_date得到合約天數membership_days

In [60]:
KKboxTransac['membership'] = KKboxTransac['mem_expire_date'] - KKboxTransac['trans_date']

In [61]:
KKboxTransac['membership_days'] = KKboxTransac['membership'] / np.timedelta64(1,'D')

##### 刪除mem_expire_date、trans_date與membership欄位

In [62]:
KKboxTransac = KKboxTransac.drop(columns=['mem_expire_date','trans_date', 'membership'])

In [63]:
KKboxTransac = KKboxTransac.sort_values(by=['msno','membership_days'])
KKboxTransac.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,membership_days
6797850,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,35,7,0,0,0,0,5.000
1521480,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,38,410,1788,1788,0,0,410.000
249714,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395,1599,1599,0,0,471.000
1914756,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,28.000
1498592,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,29.000


----

##### 在觀察KKBOX的所有方案中，最大方案天數為365天，這邊先濾去不合理的值

In [65]:
KKboxTransac = KKboxTransac[KKboxTransac["membership_days"] >= 0]

In [68]:
KKboxTransac = KKboxTransac[KKboxTransac["membership_days"] <= 365]

In [69]:
.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,membership_days
6797850,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,35,7,0,0,0,0,5.000
1914756,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,28.000
1498592,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,29.000
17923235,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,31.000
58566,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,0,31.000


In [75]:
# 存成csv
KKboxTransac.to_csv('KKboxTransac.csv',index=False,header=True)